# Домашняя работа №4. 
## Множественная проверка гипотез. Бутстрап

**Disclaimer**: 

Для каждой домашней работы обозначаются мягкие и жесткие дедлайны. За каждый день просрочки после мягкого дедлайна снимается 20% от оценки.

После жёсткого дедлайна работы не принимаются. Даже при опоздании на одну секунду. Сдавайте заранее.

Cтудент может 1 раз сдать домашнее задание после мягкого дедлайна (но до жёсткого) без штрафов.

В случае использования больших языковых моделей (large language models (LLMs), к примеру: ChatGPT, GigaChat, Qwen, etc):
- В chunk (ячейку) выше кода, созданного LLM, прикрепляйте промпт, который использовался для генерации.
- Отдельно опишите как подбирали промпты, какие заметили преимущества и недостатки GenAI для данного задания.
- За решение с указанным промптом - _штраф 40%_ для конкретного задания, который может быть пересмотрен в сторону увеличения в следующих случаях: 
    - использован ответ LLM без указания промпта _(штраф 100%)_
    - решение избыточно и, или написано неоптимально (использование magic команд без необходимости, использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, etc) _(штраф 50%)_

Также: 
- Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
- Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
- Мы в любом случае оставляем за собой право пригласить студента для защиты своего ДЗ, если заподозрим плагиат.

# Задача 1. (теория, 25 баллов)

Ответьте на следующие вопросы своими словами, сопровождая ответ примерами (примеры нужно брать не из лекции, а из собственного опыта/придумав). Половина баллов ставится за теоретический ответ, половина - за корректно приведённый пример.

* **(5 баллов)** Что такое ошибки первого и второго рода? Что такое уровень значимости теста и его мощность? Поставьте задачу проверки гипотезы (сформулируйте не только саму гипотезу, но и альтернативу). На данном примере поясните, что такое ошибка первого рода, ошибка второго рода (см. начало лекции 8 как пример).

* **(10 баллов)** Зачем нужна множественная проверка гипотез? Что такое FWER? Почему мощность метода Бонферрони меньше, чем мощность метода Холма? Приведите пример ситуации, когда применение методов множественной проверки гипотез может быть необходимо.

* **(10 баллов)**  Для чего нужен бутстреп? Какой должна быть выборка, чтобы бутстреп был успешен? Чем отличается доверительный интервал Холла от доверительного интервала Эфрона? Приведите пример небольшой выборки и 3 бутстреповских выборок из неё. Посчитайте с их помощью дисперсию первого выборочного квартиля (25% перцентиля).

In [1]:
import numpy as np
import scipy.stats as sps
import pandas as pd
import matplotlib.pyplot as plt

# Задача 2. (75 баллов)

Вы являетесь владельцем сайта, на котором размещаются аккорды и табулатуры для гитары к известным песням. Для того, чтобы извлечь прибыль из помощи начинающим музыкантам, вы размещаете на своём сайте очень релевантную пользователям рекламу. Под неё у вас имеется три выделенных места на странице сайта (то есть одновременно показываются три объявления). Если на объявление кликают, то с рекламодателя списывается определённый платёж. Каждая строчка в логе - это информация об одном заходе на сайт. В соответствующей рекламной позиции колонке пишется платёж рекламодателя, либо 0, если на рекламу пользователь не кликал.

Тут к вам приходит ML-разработчик и говорит, что он обучил модель, улучшающую ранжирование (отбор) рекламы. Вы решаете провести AB-тест. После его проведения у вас в наличии контрольная (до изменения ранжирования) и тестовая (с новой моделью ранжирования) выборки платежей рекламодателей с каждой позиции (то есть всего шесть массивов). Ваш подчинённый стажёр-аналитик уже почистил их от выбросов и загрузил их в numpy массивы. 

* **(10 баллов)** Загрузите данные. Визуализируйте их, построив сравнительные гистограммы (то есть на одном графике постройте гистограммы до и после внедрения) для каждой позиции. Какая особенность в данных имеется? Есть ли эффект от внедрения на каждой позиции или нет (визуально)?

In [2]:
data_control = pd.read_csv('control.csv', index_col=0)
data_test = pd.read_csv('test.csv', index_col=0)

In [3]:
data_control.head()

position_1  position_2  position_3
0  162.653811  129.137424  156.936867
1  141.669563   87.411934  109.776013
2    0.000000  101.022540  147.892473
3  158.722011   78.210189   97.205361
4    0.000000  177.414528    0.000000

In [4]:
data_test.head()

position_1  position_2  position_3
0    0.000000   51.098903    0.000000
1    0.000000  101.628224  278.852799
2   55.937492   81.885707  115.588688
3   46.283994    0.000000  108.444513
4    0.000000    0.000000  151.055981

* **(15 баллов)** Вы решили посмотреть на среднюю прибыль на каждой позиции в отдельности. Контролируя FWER на уровне 0.05 проверьте гипотезу о том, что внедрение увеличивает среднюю прибыль с позиции?

* **(15 баллов)** Вдруг вы вспомнили курс статистики и то, что от постановки гипотезы очень сильно зависит вывод. Контролируя FWER на уровне 0.05 проверьте, есть ли позиции, где внедрение наоборот, уменьшает прибыль?

* **(5 баллов)** Сделайте вывод о том, стоит ли выкатывать в продакшн или нет данную модель.

* **(25 баллов)** Тут вам показалось, что распределение цен не очень подходит для проверки гипотезы о средних (интересно, почему?). Да и прибыль с одной рекламной позиции вас мало интересует, интересует суммарная прибыль. 

    Постройте выборку суммарной прибыли с одного захода на сайт (для этого числа в каждой строчке нужно сложить). Постройте бутстреповский доверительный интервал Эфрона и Холла для разности медиан в контроле и тесте уровня доверия 0.95. Сравните их. На основании ДИ ответьте на вопрос: увеличивает ли значимо внедрение суммарную прибыль или нет?

* **(5 баллов)** Проверьте свой результат подходящим непараметрическим тестом из лекции 6.